In [1]:
import sys
sys.path.insert(0, '../backend/src')

from utils import s3, constants, utils

In [2]:
model = s3.load_model(constants.USGS_SITE)
archive = s3.fetch_archive_data()

WARNING - (py.warnings._showwarnmsg) - /Users/jaismith/Developer/personal/flowcast/backend/.venv/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:201: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(



In [3]:
historical = utils.prep_archive_for_training(archive)
train_df, test_df = model.split_df(historical, freq='H', valid_p=1.0 / 16)

print(train_df.shape, test_df.shape)

INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.989% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


(8714, 7) (781, 7)


In [4]:
# forecast = model.predict(train_df.drop(columns=['snow', 'snowdepth']))
forecast = model.predict(test_df.drop(columns=['snow', 'snowdepth']))

model.highlight_nth_step_ahead_of_each_forecast(72).plot(forecast, forecast_in_focus=1)
# model.plot_components(forecast)

INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.872% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H
INFO - (NP.df_utils._infer_frequency) - Major frequency H corresponds to 99.872% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - H


Predicting: 0it [00:00, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /Users/jaismith/Developer/personal/flowcast/backend/.venv/lib/python3.10/site-packages/neuralprophet/data/process.py:95: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_forecast[name] = yhat

WARNING - (py.warnings._showwarnmsg) - /Users/jaismith/Developer/personal/flowcast/backend/.venv/lib/python3.10/site-packages/neuralprophet/data/process.py:95: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_forecast[name] = yhat

WARNING - (py.warnings._showwarnmsg) - /Users/jaismith/Developer/perso

FigureWidgetResampler({
    'data': [{'fillcolor': 'rgba(45, 146, 255, 0.2)',
              'line': {'color': 'rgba(45, 146, 255, 0.2)', 'width': 1},
              'mode': 'lines',
              'name': 'yhat1 5.0%',
              'type': 'scatter',
              'uid': '427ec858-d5f3-4812-a8f1-5b1fc8516fc7',
              'x': array([datetime.datetime(2023, 8, 1, 10, 0),
                          datetime.datetime(2023, 8, 1, 11, 0),
                          datetime.datetime(2023, 8, 1, 12, 0), ...,
                          datetime.datetime(2023, 9, 2, 20, 0),
                          datetime.datetime(2023, 9, 2, 21, 0),
                          datetime.datetime(2023, 9, 2, 22, 0)], dtype=object),
              'y': array([nan, nan, nan, ..., nan, nan, nan])},
             {'fill': 'tonexty',
              'fillcolor': 'rgba(45, 146, 255, 0.2)',
              'line': {'color': 'rgba(45, 146, 255, 0.2)', 'width': 1},
              'mode': 'lines',
              'name': 'yhat1 9

In [5]:
# print(forecast)
print(model.get_latest_forecast(forecast))

                    ds          y   origin-0  origin-0 5.0%  origin-0 95.0%
0  2023-08-30 23:00:00  66.379997  65.974174      64.749153       67.029434
1  2023-08-31 00:00:00  65.839996  65.618118      64.382675       66.700523
2  2023-08-31 01:00:00  65.660004  65.476707      64.221344       66.557159
3  2023-08-31 02:00:00  65.480003  65.073761      63.726810       66.140617
4  2023-08-31 03:00:00  65.300003  65.033188      63.644035       66.021500
..                 ...        ...        ...            ...             ...
67 2023-09-02 18:00:00  66.379997  65.947899      64.180519       67.654099
68 2023-09-02 19:00:00  66.739998  66.149910      64.314171       67.981285
69 2023-09-02 20:00:00  67.279999  66.314331      64.402618       68.179718
70 2023-09-02 21:00:00  67.639999  66.401337      64.314178       68.297455
71 2023-09-02 22:00:00  67.820000  66.421616      64.311981       68.359795

[72 rows x 5 columns]
